In [1]:
import requests
import json
from psycopg2 import connect

In [24]:
# function to get review details from json file
def get_reviews(data, x):
    google_id = data['google_id']
    try:
        author_name = data['result']['reviews'][x]['author_name'].replace("'", "")
    except:
        author_name = 'no review'
    try:
        author_url = data['result']['reviews'][x]['author_url']
    except:
        author_url = 'no review'
    try:
        rating = data['result']['reviews'][x]['rating']
    except:
        rating = 'no review'
    try:
        text = data['result']['reviews'][x]['text'].replace("'", "")
    except:
        text =  'no review'
    return google_id, author_name, author_url, rating, text

In [26]:
def etl_review(restaurant):
    # search the restaurant using google api limited to London
    url = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={restaurant}%20London&inputtype=textquery&fields=place_id&key=apikey"
    
    payload={}
    headers = {}
    response = requests.get(url, headers=headers, data=payload)
    # load response into a json format
    restaurant_id = json.loads(response.text)
    # extract google_id from json file
    rest = restaurant_id['candidates'][0]['place_id']
    # use the google_id to extract information from place details api
    url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={rest}&key=apikey"
    
    payload={}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)
    # load response into a json format
    json_data = response.json()
    # add google_id into the json file
    google_id = {'google_id' : rest}
    json_data.update(google_id)
    # define the postgres table to insert the data
    table_name = 'public.google_reviews'
    # define a SQL query with variables into a string
    sql_string = f'INSERT INTO {table_name} (google_id, author_name, author_url, rating, text)\nVALUES {str([get_reviews(json_data,i) for i in range(3)]).strip("[]")};'
    # connect to the posgres database
    conn = connect(
        dbname = "initialdatabase",
        user = "luan",
        host = "demsin0166.czfwea5noxbs.eu-west-2.rds.amazonaws.com",
        password = "qwerty1234")
    # initiate a cursor
    cur = conn.cursor()
    
    try:
        # execute the query string in the database
        cur.execute( sql_string )
        conn.commit()
        print(f'{restaurant} added to database')
    except:
        print(f'{restaurant} already in database')
    # close the connection
    cur.close()
    conn.close()
    

In [22]:
list_rest = ["Bonoo Indian Tapas",
"Hibox","Scarlett Green",
"Devine Restaurant Coffee Bar",
"Indian Room",
"Nora Cafe",
"Bayleaf Restaurant",
"Amrutha Lounge",
'Andy''s Greek Taverna',
"Alexander The Great",
"Chamisse",
"Muhib Indian Cuisine",
"Spasso",
"Taste Of Nawab",
"Awesome Thai Finchley",
"Indian Moment",
"Bruce Burgers",
'Tania''s of Hampstead',
"The Rajdoot",
"Cafe Parisienne",
"The Rajdoot Hampstead",
"The Wine Place Covent Garden",
"The Park Room",
'Manuel''s Restaurant and Bar',
'Lizzie''s Cucina - Italian Restaurant',
"Dobar",
"Room 43 Bar & Kitchen",
"Paladar",
"Launceston Place",
"Eastern Eye Balti House"]

In [23]:
for i in list_rest:
    etl_review(i)

Bonoo Indian Tapas added to database
Hibox already in database
Scarlett Green added to database
Devine Restaurant Coffee Bar already in database
Indian Room added to database
Nora Cafe already in database
Bayleaf Restaurant added to database
Amrutha Lounge added to database
Andys Greek Taverna already in database
Alexander The Great already in database
Chamisse added to database
Muhib Indian Cuisine added to database
Spasso already in database
Taste Of Nawab added to database
Awesome Thai Finchley already in database
Indian Moment added to database
Bruce Burgers already in database
Tanias of Hampstead already in database
The Rajdoot added to database
Cafe Parisienne added to database
The Rajdoot Hampstead added to database
The Wine Place Covent Garden added to database
The Park Room already in database
Manuels Restaurant and Bar already in database
Lizzies Cucina - Italian Restaurant already in database
Dobar already in database
Room 43 Bar & Kitchen already in database
Paladar added t